In [1]:
import pandas as pd 
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv(r'D:\IMP  ML  PROJECTS\CAR PRICE PREDICTION\cleaned_car_dataset.csv')

In [3]:
df['Color'].value_counts()

Color
Black            5033
White            4489
Silver           3792
Grey             2375
Blue             1396
Red               639
Green             322
Orange            253
Brown             187
Carnelian red     179
Golden            145
Beige             134
Sky blue          122
Yellow            106
Purple             39
Pink               26
Name: count, dtype: int64

In [4]:
df['Category'].value_counts()

Category
Sedan          8736
Jeep           5473
Hatchback      2847
Minivan         647
Coupe           532
Universal       364
Microbus        306
Goods wagon     233
Pickup           52
Cabriolet        36
Limousine        11
Name: count, dtype: int64

In [5]:
# Handling some features columns 
df = pd.get_dummies(df, columns=['Color', 'Category'], drop_first=True)

In [6]:
req_col = []
for col in df.columns:
    if col not in ['ID', 'Levy', 'Manufacturer', 'Model', 'Doors', 'Category']: # Colors
        req_col.append(col)
req_col

['Price',
 'year',
 'Leather interior',
 'Engine volume',
 'Mileage',
 'Cylinders',
 'Wheel',
 'Airbags',
 'Engine Type',
 'Gear box type_Automatic',
 'Gear box type_Manual',
 'Gear box type_Tiptronic',
 'Gear box type_Variator',
 'Drive wheels_4WD',
 'Drive wheels_FWD',
 'Drive wheels_RWD',
 'Fuel_Diesel',
 'Fuel_Hybrid',
 'Fuel_Hydrogen',
 'Fuel_LPG',
 'Fuel_Petrol',
 'Fuel_Plug-in Hybrid',
 'Color_Black',
 'Color_Blue',
 'Color_Brown',
 'Color_Carnelian red',
 'Color_Golden',
 'Color_Green',
 'Color_Grey',
 'Color_Orange',
 'Color_Pink',
 'Color_Purple',
 'Color_Red',
 'Color_Silver',
 'Color_Sky blue',
 'Color_White',
 'Color_Yellow',
 'Category_Coupe',
 'Category_Goods wagon',
 'Category_Hatchback',
 'Category_Jeep',
 'Category_Limousine',
 'Category_Microbus',
 'Category_Minivan',
 'Category_Pickup',
 'Category_Sedan',
 'Category_Universal']

In [7]:
copy_df = df[req_col].copy()

In [8]:
copy_df

,Price,year,Leather interior,Engine volume,Mileage,Cylinders,Wheel,Airbags,Engine Type,Gear box type_Automatic,...,Category_Coupe,Category_Goods wagon,Category_Hatchback,Category_Jeep,Category_Limousine,Category_Microbus,Category_Minivan,Category_Pickup,Category_Sedan,Category_Universal
0,13328.0,2010,1,3.5,186005.0,6.0,1,12.0,0,1,...,False,False,False,True,False,False,False,False,False,False
1,16621.0,2011,0,3.0,192000.0,6.0,1,8.0,0,0,...,False,False,False,True,False,False,False,False,False,False
2,8467.0,2006,0,1.3,200000.0,4.0,0,2.0,0,0,...,False,False,True,False,False,False,False,False,False,False
3,3607.0,2011,1,2.5,168966.0,4.0,1,6.0,0,1,...,False,False,False,True,False,False,False,False,False,False
4,11726.0,2014,1,1.3,91901.0,4.0,1,4.0,0,1,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,8467.0,1999,1,2.0,300000.0,4.0,1,5.0,1,0,...,True,False,False,False,False,False,False,False,False,False
19233,15681.0,2011,1,2.4,161600.0,4.0,1,8.0,0,0,...,False,False,False,False,False,False,False,False,True,False
19234,26108.0,2010,1,2.0,116365.0,4.0,1,4.0,0,1,...,False,False,False,True,False,False,False,False,False,False
19235,5331.0,2007,1,2.0,51258.0,4.0,1,4.0,0,1,...,False,False,False,True,False,False,False,False,False,False


In [9]:
# Split and fit it into decision tree model 
x_train, x_test, y_train, y_test = train_test_split(copy_df.drop('Price', axis=1), copy_df['Price'], test_size=0.2, random_state=42)

In [10]:
# Define the parameters you want to test
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1]
}

# Initialize GridSearchCV with the Regressor
grid_search = GridSearchCV(
    estimator=xgb.XGBRegressor(objective='reg:squarederror'),
    param_grid=param_grid,
    cv=3,             # Splits data into 3 parts to validate
    scoring='neg_mean_squared_error',
    verbose=1
)

# Run the search
grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","XGBRegressor(...ree=None, ...)"
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'learning_rate': [0.01, 0.1], 'max_depth': [3, 5, ...], 'n_estimators': [100, 200]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'neg_mean_squared_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",3
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is d

In [11]:
# Generate price predictions for the test set
y_pred = grid_search.predict(x_test)

print("R2 Score : ", r2_score(y_test, y_pred))

R2 Score :  0.646653446152146


In [14]:
import joblib

# Extract the best model from the grid search
best_model = grid_search.best_estimator_

# Save the model with the .joblib extension
joblib.dump(best_model, 'car_price_model.joblib')

['car_price_model.joblib']

In [13]:
df

,ID,Price,Levy,Manufacturer,Model,year,Leather interior,Engine volume,Mileage,Cylinders,...,Category_Coupe,Category_Goods wagon,Category_Hatchback,Category_Jeep,Category_Limousine,Category_Microbus,Category_Minivan,Category_Pickup,Category_Sedan,Category_Universal
0,45654403,13328.0,1399,LEXUS,RX 450,2010,1,3.5,186005.0,6.0,...,False,False,False,True,False,False,False,False,False,False
1,44731507,16621.0,1018,CHEVROLET,Equinox,2011,0,3.0,192000.0,6.0,...,False,False,False,True,False,False,False,False,False,False
2,45774419,8467.0,-,HONDA,FIT,2006,0,1.3,200000.0,4.0,...,False,False,True,False,False,False,False,False,False,False
3,45769185,3607.0,862,FORD,Escape,2011,1,2.5,168966.0,4.0,...,False,False,False,True,False,False,False,False,False,False
4,45809263,11726.0,446,HONDA,FIT,2014,1,1.3,91901.0,4.0,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,45798355,8467.0,-,MERCEDES-BENZ,CLK 200,1999,1,2.0,300000.0,4.0,...,True,False,False,False,False,False,False,False,False,False
19233,45778856,15681.0,831,HYUNDAI,Sonata,2011,1,2.4,161600.0,4.0,...,False,False,False,False,False,False,False,False,True,False
19234,45804997,26108.0,836,HYUNDAI,Tucson,2010,1,2.0,116365.0,4.0,...,False,False,False,True,False,False,False,False,False,False
19235,45793526,5331.0,1288,CHEVROLET,Captiva,2007,1,2.0,51258.0,4.0,...,False,False,False,True,False,False,False,False,False,False
